# L1: Automated Project: Planning, Estimation, and Allocation

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Set OpenAI Model

In [13]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [14]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output

In [15]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [16]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

2025-08-20 05:03:17,937 - 139902151981952 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Crew's Inputs

In [17]:
from IPython.display import display, Markdown

project = 'Online Stock Trading Platform'
industry = 'Fintech'
project_objectives = 'Develop a secure, high-performance online platform for stock trading and portfolio management'
team_members = """
- Alex Chen (Project Manager)
- Maria Rodriguez (Solutions Architect)
- David Kim (Full-Stack Engineer)
- Emily White (Backend Engineer)
- Sarah Lee (Frontend Engineer)
- Ben Carter (QA Engineer)
- Jessica Green (DevOps Engineer)
- Mike Johnson (UI/UX Designer)
- Chloe Davis (Compliance & Legal Specialist)
"""
project_requirements = """
- **Requirement Analysis & Planning:**
    - Define user personas (e.g., Novice Investor, Day Trader).
    - Comply with financial regulations (e.g., KYC, AML) and data privacy laws (e.g., GDPR).
    - Specify functional requirements: real-time market data, various order types (market, limit), portfolio tracking, and secure account funding.
    - Set non-functional requirements: low-latency performance, high security (encryption, MFA), and scalability.
- **Architecture & Design:**
    - Design a microservices architecture for independent, scalable components (e.g., Order Service, User Service).
    - Implement a real-time data processing pipeline using message queues (e.g., Kafka) and WebSockets for client updates.
    - Design a robust database schema optimized for high-volume transactions and real-time data retrieval.
    - Plan for cloud deployment on a major provider (e.g., AWS, GCP) using services like Kubernetes for orchestration.
- **Development:**
    - **Front-end:** Build an intuitive trading dashboard with real-time charts, order forms, and portfolio views.
    - **Back-end:** Develop a reliable order matching engine, integrate with market data APIs, and implement risk management logic.
    - Implement secure user authentication (including MFA) and API gateways.
- **Testing:**
    - Conduct comprehensive unit, integration, and end-to-end testing for all services.
    - Perform performance and load testing to simulate high trading volumes and ensure system stability.
    - Execute security testing (penetration testing, vulnerability scans) to protect user data and assets.
    - Conduct User Acceptance Testing (UAT) with a select group of users.
- **CI/CD & Deployment:**
    - Set up a Continuous Integration/Continuous Deployment (CI/CD) pipeline using tools like GitLab CI or GitHub Actions.
    - Use Docker for containerization and Kubernetes for automated deployment and scaling.
    - Implement a monitoring and logging solution (e.g., Prometheus, Grafana) to track system health in production.
- **Production & Maintenance:**
    - Deploy the platform to a secure cloud environment.
    - Establish a disaster recovery plan to ensure business continuity.
    - Set up an ongoing maintenance schedule for security patches, software updates, and new feature releases.
    - Provide a robust customer support system for user inquiries and technical issues.
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Online Stock Trading Platform

**Project Objectives:** Develop a secure, high-performance online platform for stock trading and portfolio management

**Industry:** Fintech

**Team Members:**

- Alex Chen (Project Manager)
- Maria Rodriguez (Solutions Architect)
- David Kim (Full-Stack Engineer)
- Emily White (Backend Engineer)
- Sarah Lee (Frontend Engineer)
- Ben Carter (QA Engineer)
- Jessica Green (DevOps Engineer)
- Mike Johnson (UI/UX Designer)
- Chloe Davis (Compliance & Legal Specialist)

**Project Requirements:**

- **Requirement Analysis & Planning:**
    - Define user personas (e.g., Novice Investor, Day Trader).
    - Comply with financial regulations (e.g., KYC, AML) and data privacy laws (e.g., GDPR).
    - Specify functional requirements: real-time market data, various order types (market, limit), portfolio tracking, and secure account funding.
    - Set non-functional requirements: low-latency performance, high security (encryption, MFA), and scalability.
- **Architecture & Design:**
    - Design a microservices architecture for independent, scalable components (e.g., Order Service, User Service).
    - Implement a real-time data processing pipeline using message queues (e.g., Kafka) and WebSockets for client updates.
    - Design a robust database schema optimized for high-volume transactions and real-time data retrieval.
    - Plan for cloud deployment on a major provider (e.g., AWS, GCP) using services like Kubernetes for orchestration.
- **Development:**
    - **Front-end:** Build an intuitive trading dashboard with real-time charts, order forms, and portfolio views.
    - **Back-end:** Develop a reliable order matching engine, integrate with market data APIs, and implement risk management logic.
    - Implement secure user authentication (including MFA) and API gateways.
- **Testing:**
    - Conduct comprehensive unit, integration, and end-to-end testing for all services.
    - Perform performance and load testing to simulate high trading volumes and ensure system stability.
    - Execute security testing (penetration testing, vulnerability scans) to protect user data and assets.
    - Conduct User Acceptance Testing (UAT) with a select group of users.
- **CI/CD & Deployment:**
    - Set up a Continuous Integration/Continuous Deployment (CI/CD) pipeline using tools like GitLab CI or GitHub Actions.
    - Use Docker for containerization and Kubernetes for automated deployment and scaling.
    - Implement a monitoring and logging solution (e.g., Prometheus, Grafana) to track system health in production.
- **Production & Maintenance:**
    - Deploy the platform to a secure cloud environment.
    - Establish a disaster recovery plan to ensure business continuity.
    - Set up an ongoing maintenance schedule for security patches, software updates, and new feature releases.
    - Provide a robust customer support system for user inquiries and technical issues.



## Kicking off the crew

In [18]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Online Stock Trading Platform project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- **Requirement Analysis & Planning:**
    - Define user personas (e.g., Novice Investor, Day Trader).
    - Comply with financial regulations (e.g., KYC, AML) and data privacy laws (e.g., GDPR).
    - Specify functional requirements: real-time market data, various order types (market, limit), portfolio tracking, and secure account funding.
    - Set non-functional requirements: low-latency performance, high security (encryption, MFA), and scalability.
- **Architecture & Design:**
    - Design a microservices architecture for independent, scalable components (e.g., Order Service, User Service).
    - Implement a real-time data processing pipeline using message queues (e.g., Kafka) and WebSockets f

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [19]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0037


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,24691,14695,9996,9


## Result

In [20]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Define User Personas',
   'estimated_time_hours': 40.0,
   'required_resources': ['Mike Johnson']},
  {'task_name': 'Comply with Financial Regulations',
   'estimated_time_hours': 80.0,
   'required_resources': ['Chloe Davis']},
  {'task_name': 'Specify Functional Requirements',
   'estimated_time_hours': 80.0,
   'required_resources': ['Maria Rodriguez', 'Mike Johnson']},
  {'task_name': 'Set Non-Functional Requirements',
   'estimated_time_hours': 40.0,
   'required_resources': ['Maria Rodriguez']},
  {'task_name': 'Design Microservices Architecture',
   'estimated_time_hours': 80.0,
   'required_resources': ['Maria Rodriguez']},
  {'task_name': 'Implement Real-Time Data Processing Pipeline',
   'estimated_time_hours': 120.0,
   'required_resources': ['Emily White', 'Jessica Green']},
  {'task_name': 'Design Database Schema',
   'estimated_time_hours': 40.0,
   'required_resources': ['Maria Rodriguez', 'David Kim']},
  {'task_name': 'Plan for Cloud Deploymen

## Inspect further

In [21]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Define User Personas,40.000000,['Mike Johnson']
1,Comply with Financial Regulations,80.000000,['Chloe Davis']
2,Specify Functional Requirements,80.000000,"['Maria Rodriguez', 'Mike Johnson']"
3,Set Non-Functional Requirements,40.000000,['Maria Rodriguez']
4,Design Microservices Architecture,80.000000,['Maria Rodriguez']
5,Implement Real-Time Data Processing Pipeline,120.000000,"['Emily White', 'Jessica Green']"
6,Design Database Schema,40.000000,"['Maria Rodriguez', 'David Kim']"
7,Plan for Cloud Deployment,40.000000,['Jessica Green']
8,Front-end Development,160.000000,['Sarah Lee']
9,Back-end Development,160.000000,"['David Kim', 'Emily White']"


### Inspecting Milestones

In [22]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Requirement Analysis & Planning,"['Define User Personas', 'Comply with Financial Regulations', 'Specify Functional Requirements', 'Set Non-Functional Requirements']"
1,Architecture & Design,"['Design Microservices Architecture', 'Implement Real-Time Data Processing Pipeline', 'Design Database Schema', 'Plan for Cloud Deployment']"
2,Development,"['Front-end Development', 'Back-end Development', 'Secure User Authentication']"
3,Testing,"['Unit Testing', 'Performance and Load Testing', 'Security Testing', 'User Acceptance Testing (UAT)']"
4,CI/CD & Deployment,"['CI/CD Pipeline Setup', 'Containerization with Docker', 'Monitoring & Logging Solution']"
5,Production & Maintenance,"['Final Deployment', 'Disaster Recovery Plan', 'Ongoing Maintenance Schedule', 'Customer Support System']"
